In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
import argparse
import collections
import cupy as xp
import numpy as np
import six

import chainer
from chainer.backends import cuda
import chainer.functions as F
import chainer.initializers as I
import chainer.links as L
import chainer.optimizers as O
from chainer import reporter
from chainer import training
from chainer.training import extensions
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import pyodbc
import pandas as pd
from multiprocessing import Pool
import re
import pickle
import time
import warnings

warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
warnings.filterwarnings(action='ignore', category=FutureWarning, module='cupy')
from gensim import corpora
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }
class Parallel_process_source_dic():
    def __init__(self):
        pass

    # ここを並列処理する
    def func(document, argument1):
        wrk_source = list(map(argument1.get, document))
        return wrk_source

    def wrapper(args):
        return Parallel_process_source_dic.func(*args)

    def multi_process(proc_list):
        # プロセス数:8(8個のcpuで並列処理)
        p = Pool(6)
        output = p.map(Parallel_process_source_dic.wrapper, proc_list)
        # プロセスの終了
        p.close()
        return output

    def main_proc_1(source, dictionary):
        proc_list = [(item, dictionary) for item in source]

        output = Parallel_process_source_dic.multi_process(proc_list)
        return output
class WindowIterator(chainer.dataset.Iterator):
    """Dataset iterator to create a batch of sequences at different positions.

    This iterator returns a pair of the current words and the context words.
    """

    def __init__(self,
                 small_str_source=None,
                 small_int_source=None,
                 big_str_source = None,
                 big_int_source = None,
                 float_source=None,
                 ans=None,
                batch_size=None, 
                repeat=True): 
        self.small_str_source = small_str_source
        self.small_int_source = small_int_source
        self.big_str_source = big_str_source
        self.big_int_source = big_int_source
        self.float_source = float_source
        self.ans = ans 
        self.batch_size = batch_size
        self._repeat = repeat
        # order is the array which is shuffled ``[window, window + 1, ...,
        # len(dataset) - window - 1]``
        
        self.current_position = 0
        # Number of completed sweeps over the dataset. In this case, it is
        # incremented if every word is visited at least once after the last
        # increment.
        self.epoch = 0
        # True if the epoch is incremented at the last iteration.
        self.is_new_epoch = False

    def __next__(self):
        """This iterator returns a list representing a mini-batch.

        Each item indicates a different position in the original sequence.
        """
        if not self._repeat and self.epoch > 0:
            raise StopIteration

        i = self.current_position
        i_end = i + self.batch_size
        mini_small_data_str = self.small_str_source[i:i_end]
        mini_small_data_int = self.small_int_source[i:i_end]
        mini_big_data_str = self.big_str_source[:,i:i_end]
        mini_big_data_int = self.big_int_source[:,i:i_end]
        mini_data_float = self.float_source[i:i_end]
        ans_data = self.ans[i:i_end] 
        
        if i_end >= len(self.small_str_source):
            self.epoch += 1
            self.is_new_epoch = True
            self.current_position = 0
        else:
            self.is_new_epoch = False
            self.current_position = i_end

        return mini_small_data_str,mini_small_data_int,mini_big_data_str,mini_big_data_int,mini_data_float,ans_data

    @property
    def epoch_detail(self):
        return self.epoch + float(self.current_position) / len(self.small_str_source)

    def serialize(self, serializer):
        self.current_position = serializer('current_position',
                                           self.current_position)
        self.epoch = serializer('epoch', self.epoch)
        self.is_new_epoch = serializer('is_new_epoch', self.is_new_epoch)
        if self._order is not None:
            serializer('_order', self._order)
def convert(batch, device):
    mini_small_data_str,mini_small_data_int,mini_big_data_str,mini_big_data_int,mini_data_float,ans_data = batch
    #if device >= 0:
    #    mini_data_str = cuda.to_gpu(mini_data_str)
    #    mini_data_int = cuda.to_gpu(mini_data_float)
    #    mini_data_float = cuda.to_gpu(mini_data_float)
    #    ans_data = cuda.to_gpu(ans_data) 
    return mini_small_data_str,mini_small_data_int,mini_big_data_str,mini_big_data_int,mini_data_float,ans_data

class ms_malware(Chain):
    """Definition of Skip-gram Model""" 
    def __init__(self, 
                 small_str_source_len,
                 small_int_source_len,
                 big_str_source_len,
                 big_int_source_len,
                 float_source_len,
                 small_str_dictionary,
                 small_int_dictionary,
                 big_str_dictionary,
                 big_int_dictionary):
        super(ms_malware, self).__init__()

        with self.init_scope():
            self.small_embed_str = L.EmbedID(int(len(small_str_dictionary.items())) + 1,300,ignore_label =-1)
            self.small_linear_str1 = L.Linear(small_str_source_len * 300,100)
            #self.small_batch_str = L.BatchNormalization(500)
             
            self.small_embed_int = L.EmbedID(int(len(small_int_dictionary.items())) + 1,300,ignore_label =-1)
            self.small_linear_int1 = L.Linear(small_int_source_len * 300,100)
            #self.small_batch_int = L.BatchNormalization(500)
            for i in range(big_str_source_len): 
                self.add_link('embed_str' + str(i),L.EmbedID(int(len(big_str_dictionary[i].items())) + 1,300,ignore_label =-1))
                self.add_link('linear_str' + str(i),L.Linear(300,100))
                #self.add_link('linear_str'+str(i),L.Linear((int(len(str_dictionary[i].items()))*
                #100),50))
                #self.add_link('batch_str' + str(i), L.BatchNormalization(500))
            for i in range(big_int_source_len): 
                self.add_link('embed_int' + str(i),L.EmbedID(int(len(big_int_dictionary[i].items())) + 1,300,ignore_label =-1))
                self.add_link('linear_int' + str(i),L.Linear(300,100))
                #self.add_link('batch_int' + str(i), L.BatchNormalization(500))


            self.linear_float = L.Linear(float_source_len,100)
            #self.batch__float = L.BatchNormalization(500)

            self.L1 = L.Linear(big_int_source_len*100 +big_str_source_len * 100 + 100 + 100 + 100 , 4000)
            self.L2 = L.Linear(4000, 4000)
            self.L3 = L.Linear(4000, 2)
            self.small_str_source_len = small_str_source_len
            self.small_int_source_len = small_int_source_len
            self.big_str_source_len = big_str_source_len
            self.big_int_source_len = big_int_source_len
            self.float_source_len = float_source_len
    def forward(self,mini_small_data_str,mini_small_data_int,mini_big_data_str,mini_big_data_int,mini_data_float,ans_data):
        mini_small_data_str = xp.array(mini_small_data_str,dtype=xp.int32)
        mini_small_data_int = xp.array(mini_small_data_int,dtype=xp.int32)
        mini_big_data_str = xp.array(mini_big_data_str,dtype=xp.int32)
        mini_big_data_int = xp.array(mini_big_data_int,dtype=xp.int32)
        mini_data_float = xp.array(mini_data_float,dtype=xp.float32)
        ys = xp.array(ans_data,dtype=xp.int32)

        h1 = Variable()
        h2 = Variable()
        h3 = Variable()
        for i in range(self.big_str_source_len): 
            if i == 0:
                em2h_0 = self['embed_str' + str(i)](mini_big_data_str[i])
                h1 = self['linear_str' + str(i)](em2h_0)
                #h1 = F.relu(self['batch_str' + str(i)](em2h_1))
                
            else:
                em2h_0 = self['embed_str' + str(i)](mini_big_data_str[i])
                em2h_2 = self['linear_str' + str(i)](em2h_0)
                #em2h_2 = F.relu(self['batch_str' + str(i)](em2h_1))
                h1 = F.concat((h1, em2h_2), axis=1)
        for i in range(self.big_int_source_len): 
            if i == 0:
                em2h_0 = self['embed_int' + str(i)](mini_big_data_int[i])
                h2 = self['linear_int' + str(i)](em2h_0)
                #h2 = F.relu(self['batch_int' + str(i)](em2h_1))
                
            else:
                em2h_0 = self['embed_int' + str(i)](mini_big_data_int[i])
                em2h_2 = self['linear_int' + str(i)](em2h_0)
                #em2h_2 = F.relu(self['batch_int' + str(i)](em2h_1))
                h2 = F.concat((h2, em2h_2), axis=1)
            pass
        h1_1 = self.small_embed_str(mini_small_data_str)
        h1_1 = F.relu(self.small_linear_str1(h1_1) )
        #h1_1 = F.relu(self.small_batch_str(h1_1)) 

        h2_1 = self.small_embed_int(mini_small_data_int)
        h2_1 = F.relu(self.small_linear_int1(h2_1)) 
        #h2_1 = F.relu(self.small_batch_int(h2_1)) 

        h3 = self.linear_float(mini_data_float)
        #h3 = self.batch__float(h3)

        h4 = F.concat((h1, h2), axis=1) 
        h4 = F.concat((h4, h1_1), axis=1)
        h4 = F.concat((h4, h2_1), axis=1)
        h4 = F.concat((h4, h3), axis=1)
        h5 = self.L1(h4)
        h5 =  self.L2(h5)
        y = self.L3(h5)
        loss = F.softmax_cross_entropy(y, ys) 
        acc = F.accuracy(y, ys)
        report({'accuracy': acc.data}, self)
        report({'loss': loss.data}, self)
        return loss  
#https://www.kaggle.com/theoviel/load-the-totality-of-the-data


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
def non_cache():
    dir1 = '../input/vvwwww1/'
    project_name = 'proc2_'
    print("start training")
    floats = ['float16', 'float32', 'float64']
    ints = ['int8', 'int16', 'int32', 'int64']
    category = ['float16', 'float32', 'float64','int8', 'int16', 'int32', 'int64']
    float_columns = [c for c,v in dtypes.items() if v in floats]
    int_columns = [c for c,v in dtypes.items() if v in ints]
    categorical_columns = [c for c,v in dtypes.items() if v not in category]

    proc_data = pd.read_csv(r"U:\ms\train.csv")  
    proc_data = proc_data.drop('MachineIdentifier', axis=1)
    y = proc_data['HasDetections'].values.tolist()  
    proc_data = proc_data.drop('HasDetections', axis=1) 
    proc_data = reduce_mem_usage(proc_data)


    stats = []
    for col in proc_data.columns:
        stats.append((col, proc_data[col].nunique(), proc_data[col].isnull().sum() * 100 / proc_data.shape[0], proc_data[col].value_counts(normalize=True, dropna=False).values[0] * 100, proc_data[col].dtype))
    
    stats_df = pd.DataFrame(stats, columns=['Feature', 'Unique_values', 'Percentage of missing values', 'Percentage of values in the biggest category', 'type'])
    stats_df.sort_values('Percentage of missing values', ascending=False)
 
    good_cols = list(proc_data.columns)
    for col in proc_data.columns:
        rate = proc_data[col].value_counts(normalize=True, dropna=False).values[0]
        if rate > 0.9:
            good_cols.remove(col)
    float_columns1 = []
    int_columns1 = []
    categorical_columns1 = []
    for item in float_columns:
        if item in good_cols:
            float_columns1.append(item)
        pass
    for item in int_columns:
        if item  in good_cols:
            int_columns1.append(item)
        pass
    for item in categorical_columns:
        if item in good_cols:
            categorical_columns1.append(item)
            
    small_int_columns = []
    small_str_columns = []
    big_int_columns = []
    big_str_columns = []

    for item in int_columns1:
        if stats_df[stats_df.Feature ==item]['Unique_values'].values > 100:
            big_int_columns.append(item)
        else:
            small_int_columns.append(item)
    for item in categorical_columns1:
        if stats_df[stats_df.Feature ==item]['Unique_values'].values > 100:
            big_str_columns.append(item)
        else:
            small_str_columns.append(item)

    proc_data = proc_data[good_cols] 

    small_str_source =[item1 for item1 in proc_data[small_str_columns].fillna('-1').values.tolist()]
    small_str_dictionary = corpora.Dictionary(small_str_source) 
    small_str_dictionary = dict([[key, value1 ] for key, value1 in small_str_dictionary.token2id.items()])
    small_str_dictionary['-1'] = -1
    small_str_source =Parallel_process_source_dic.main_proc_1(small_str_source,small_str_dictionary)

    big_str_dictionary =[]
    str_int = -1
    big_str_source =[]
    big_str_source =[]
    for item in big_str_columns:
        str_int = str_int +1
        source = [[item1] for item1 in proc_data[item].fillna('-1').values.tolist()]
        big_str_dictionary.append(corpora.Dictionary(source))
        big_str_dictionary[str_int] = dict([[key, value1 ] for key, value1 in big_str_dictionary[str_int].token2id.items()])
        big_str_dictionary[str_int]['-1'] = -1
        big_str_source.append(Parallel_process_source_dic.main_proc_1(source,big_str_dictionary[str_int]))
        print(item +':' +str(len(big_str_dictionary[str_int].items())))

    small_int_source =[[str(v) for v in item1] for item1 in proc_data[small_int_columns].fillna('-1').values.tolist()]
    small_int_dictionary = corpora.Dictionary(small_int_source) 
    small_int_dictionary = dict([[key, value1 ] for key, value1 in small_int_dictionary.token2id.items()])
    small_int_dictionary['-1'] = -1
    small_int_source =Parallel_process_source_dic.main_proc_1(small_int_source,small_int_dictionary)

    big_int_dictionary =[]
    int_int = -1
    big_int_source =[]
    for item in big_int_columns:
        int_int = int_int +1
        source = [[str(item1)] for item1 in proc_data[item].fillna('-1').values.tolist()]
        big_int_dictionary.append(corpora.Dictionary(source))
        big_int_dictionary[int_int] = dict([[key, value1 ] for key, value1 in big_int_dictionary[int_int].token2id.items()])
        big_int_dictionary[int_int]['-1'] = -1
        big_int_source.append(Parallel_process_source_dic.main_proc_1(source,big_int_dictionary[int_int]))
        print(item +':'+str(len(big_int_dictionary[int_int].items())))
    float_source =[] 
   
    float_source = [item1 for item1 in proc_data[float_columns1].fillna(0).values.tolist()]
        
    
    small_str_source_len = len(small_str_source[0])
    big_str_source_len = len(big_str_source)
    small_int_source_len = len(small_int_source[0])
    big_int_source_len = len(big_int_source)
    float_source_len =len(float_source[0])

    small_str_source = np.array(small_str_source,dtype=np.int32)
    small_int_source = np.array(small_int_source,dtype=np.int32)
    big_str_source = np.array(big_str_source,dtype=np.int32)
    big_int_source = np.array(big_int_source,dtype=np.int32)
    float_source = np.array(float_source,dtype=np.float32)
    with open(dir1 + project_name + 'small_str_dictionary.pickle', 'wb') as f:  # save cache
        pickle.dump(small_str_dictionary, f)
    with open(dir1 + project_name + 'small_int_dictionary.pickle', 'wb') as f:  # save cache
        pickle.dump(small_int_dictionary, f)
    with open(dir1 + project_name + 'big_str_dictionary.pickle', 'wb') as f:  # save cache
        pickle.dump(big_str_dictionary, f)
    with open(dir1 + project_name + 'big_int_dictionary.pickle', 'wb') as f:  # save cache
        pickle.dump(big_int_dictionary, f)
    #len
    with open(dir1 + project_name + 'small_str_source_len.pickle', 'wb') as f:  # save cache
        pickle.dump(small_str_source_len, f)

    with open(dir1 + project_name + 'big_str_source_len.pickle', 'wb') as f:  # save cache
        pickle.dump(big_str_source_len, f)

    with open(dir1 + project_name + 'small_int_source_len.pickle', 'wb') as f:  # save cache
        pickle.dump(small_int_source_len, f)

    with open(dir1 + project_name + 'big_int_source_len.pickle', 'wb') as f:  # save cache
        pickle.dump(big_int_source_len, f)

    with open(dir1 + project_name + 'float_source_len.pickle', 'wb') as f:  # save cache
        pickle.dump(float_source_len, f)
    #data
    with open(dir1 + project_name + 'small_str_source.pickle', 'wb') as f:  # save cache
        pickle.dump(small_str_source, f)

    with open(dir1 + project_name + 'small_int_source.pickle', 'wb') as f:  # save cache
        pickle.dump(small_int_source, f)

    with open(dir1 + project_name + 'big_str_source.pickle', 'wb') as f:  # save cache
        pickle.dump(big_str_source, f)

    with open(dir1 + project_name + 'big_int_source.pickle', 'wb') as f:  # save cache
        pickle.dump(big_int_source, f)

    with open(dir1 + project_name + 'float_source.pickle', 'wb') as f:  # save cache
        pickle.dump(float_source, f)
    with open(dir1 + project_name + 'y.pickle', 'wb') as f:  # save cache
        pickle.dump(y, f)

    model = ms_malware(small_str_source_len,
                        small_int_source_len,
                        big_str_source_len,
                        big_int_source_len,
                        float_source_len,
                        small_str_dictionary,
                        small_int_dictionary,
                        big_str_dictionary,
                        big_int_dictionary)
    
    if gpu >= 0:
        model.to_gpu()

    # Set up an optimizer
    optimizer = O.Adam()
    optimizer.setup(model)
    batchsize = 2046
    epoch =20
    gpu = 0
    # Set up an iterator
    train_iter = WindowIterator(small_str_source=small_str_source,
                                 small_int_source=small_int_source,
                                 big_str_source = big_str_source,
                                 big_int_source = big_int_source,
                                 float_source=float_source,
                                 ans=y,
                                batch_size=batchsize, 
                                repeat=True)
    
    # Set up an updater
    updater = training.updaters.StandardUpdater(
        train_iter, optimizer, converter=convert, device=gpu)

    # Set up a trainer
    trainer = training.Trainer(updater, (epoch, 'epoch'), out="result")
    trainer.extend(extensions.LogReport())
    trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'validation/main/loss', 'main/accuracy', 'validation/main/accuracy']))
    trainer.extend(extensions.ProgressBar())
    trainer.run()
    model.to_cpu()
    serializers.save_npz(dir1 + project_name + '.model', model)
    serializers.save_npz(dir1 + project_name + '.state', optimizer)
    pass
def yes_cache():
    gpu = 0
    dir1 = '../input/vvwwww1/'
    project_name = 'proc2_'
    with open(dir1 + project_name + 'small_str_dictionary.pickle', 'rb') as f: #load cache
        small_str_dictionary = pickle.load(f)
    with open(dir1 + project_name + 'small_int_dictionary.pickle', 'rb') as f: #load cache
        small_int_dictionary = pickle.load(f)
    with open(dir1 + project_name + 'big_str_dictionary.pickle', 'rb') as f: #load cache
        big_str_dictionary = pickle.load(f)
    with open(dir1 + project_name + 'big_int_dictionary.pickle', 'rb') as f: #load cache
        big_int_dictionary = pickle.load(f)
    #len

    with open(dir1 + project_name + 'small_str_source_len.pickle', 'rb') as f: #load cache
        small_str_source_len = pickle.load(f)
    with open(dir1 + project_name + 'big_str_source_len.pickle', 'rb') as f: #load cache
        big_str_source_len = pickle.load(f)
    with open(dir1 + project_name + 'small_int_source_len.pickle', 'rb') as f: #load cache
        small_int_source_len = pickle.load(f)
    with open(dir1 + project_name + 'big_int_source_len.pickle', 'rb') as f: #load cache
        big_int_source_len = pickle.load(f)
    with open(dir1 + project_name + 'float_source_len.pickle', 'rb') as f: #load cache
        float_source_len = pickle.load(f)
    #data  
    with open(dir1 + project_name + 'small_str_source.pickle', 'rb') as f: #load cache
        small_str_source = pickle.load(f)
    with open(dir1 + project_name + 'small_int_source.pickle', 'rb') as f: #load cache
        small_int_source = pickle.load(f)
    with open(dir1 + project_name + 'big_str_source.pickle', 'rb') as f: #load cache
        big_str_source = pickle.load(f)
    with open(dir1 + project_name + 'big_int_source.pickle', 'rb') as f: #load cache
        big_int_source = pickle.load(f)
    with open(dir1 + project_name + 'small_str_source.pickle', 'rb') as f: #load cache
        small_str_source = pickle.load(f)
    with open(dir1 + project_name + 'float_source.pickle', 'rb') as f: #load cache
        float_source = pickle.load(f)
    with open(dir1 + project_name + 'y.pickle', 'rb') as f: #load cache
        y = pickle.load(f)
         
    model = ms_malware(small_str_source_len,
                        small_int_source_len,
                        big_str_source_len,
                        big_int_source_len,
                        float_source_len,
                        small_str_dictionary,
                        small_int_dictionary,
                        big_str_dictionary,
                        big_int_dictionary )
    
    if gpu >= 0:
       model.to_gpu()

    # Set up an optimizer
    optimizer = O.Adam()
    optimizer.setup(model)
    batchsize =2046
    epoch =20
    gpu = 0
    # Set up an iterator
    train_iter = WindowIterator(small_str_source=small_str_source,
                                 small_int_source=small_int_source,
                                 big_str_source = big_str_source,
                                 big_int_source = big_int_source,
                                 float_source=float_source,
                                 ans=y,
                                batch_size=batchsize, 
                                repeat=True)
    
    # Set up an updater
    updater = training.updaters.StandardUpdater(
        train_iter, optimizer, converter=convert, device=gpu)

    # Set up a trainer
    trainer = training.Trainer(updater, (epoch, 'epoch'), out="result")
    trainer.extend(extensions.LogReport())
    trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'validation/main/loss', 'main/accuracy', 'validation/main/accuracy']))
    #trainer.extend(extensions.ProgressBar())
    trainer.run()
    model.to_cpu()
    serializers.save_npz('1.model', model)
    serializers.save_npz('1.state', optimizer)
    pass
if __name__ =='__main__':
    #non_cache()
    yes_cache()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy
1           1.01903e+08                        0.502502                                 
2           8.88209e+07                        0.531692                                 
3           2.62078e+08                        0.547198                                 
4           6.44799e+08                        0.551514                                 
5           1.39459e+09                        0.554123                                 
6           1.77717e+09                        0.553849                                 
7           2.26297e+09                        0.554112                                 
8           1.0153e+09                        0.556173                                 
9           2.71985e+09                        0.556888                                 
10          2.00022e+09                        0.549765                                 
11          4.17282e+09  